# Projeto Final - Codenation Data Science - JOINVILLE

# SQUAD 1 - Objetivo

O objetivo deste produto é fornecer um serviço automatizado que recomenda leads para um usuário dado sua atual lista de clientes (Portfólio).

# Contextualização
Algumas empresas gostariam de saber quem são as demais empresas em um determinado mercado (população) que tem maior probabilidade se tornarem seus próximos clientes. Ou seja, a solução deve encontrar no mercado quem são os leads mais aderentes dado as características dos clientes presentes no portfólio do usuário.

Além disso, a solução deve ser agnóstica ao usuário. Qualquer usuário com uma lista de clientes que queira explorar esse mercado pode extrair valor do serviço.

Para o desafio, deverão ser consideradas as seguintes bases:

Mercado: Base com informações sobre as empresas do Mercado a ser considerado. Portfolio 1: Ids dos clientes da empresa 1 Portfolio 2: Ids dos clientes da empresa 2 Portfolio 3: Ids dos clientes da empresa 3

Obs: todas as empresas(ids) dos portfolios estão contidos no Mercado(base de população).

As bases de portfólio poderão ser utilizadas para testar a aderência da solução. Além disso, se a equipe desejar, poderá simular portfólios por meio de amostragens no mercado.

Premissas:

1. Não serão considerados os dados de "coligados"
2. Não serão considerado o relacionamento entre filial/matriz, cada cliente será avaliado individualmente

In [1]:
#Importando as bibliotecas
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns

In [2]:
from IPython.core.pylabtools import figsize

figsize(12, 8)

sns.set()

## Importando os Arquivos

In [3]:
#Importando o Dataframe de Mercado e eliminando os dados 
mercado_orig = pd.read_csv('estaticos_market.csv')
mercado = mercado_orig.copy()

mercado = mercado.drop(['Unnamed: 0'] , axis='columns')
mercado = mercado.drop(['fl_antt'] , axis='columns')
mercado = mercado.drop(['vl_total_tancagem'] , axis='columns')
mercado = mercado.drop(['de_faixa_faturamento_estimado'] , axis='columns')
mercado = mercado.drop(['vl_frota'] , axis='columns')

# Não serão considerados os dados relacionados as contratações dos funcionarios no periodo - Apenas a quantidade atual de funcionários da empresa
mercado = mercado.drop(['meses_ultima_contratacaco'] , axis='columns')
mercado = mercado.drop(['qt_admitidos_12meses'] , axis='columns')
mercado = mercado.drop(['qt_desligados_12meses'] , axis='columns')
mercado = mercado.drop(['qt_desligados'] , axis='columns')
mercado = mercado.drop(['qt_admitidos'] , axis='columns')
mercado = mercado.drop(['qt_funcionarios_12meses'] , axis='columns')
mercado = mercado.drop(['qt_funcionarios_24meses'] , axis='columns')
mercado = mercado.drop(['tx_crescimento_12meses'] , axis='columns')
mercado = mercado.drop(['tx_crescimento_24meses'] , axis='columns')
mercado = mercado.drop(['tx_rotatividade'] , axis='columns')
mercado = mercado.drop(['min_meses_servicos'] , axis='columns')
mercado = mercado.drop(['min_meses_servicos_all'] , axis='columns')
mercado = mercado.drop(['max_meses_servicos_all'] , axis='columns')
mercado = mercado.drop(['grau_instrucao_macro_desconhecido'] , axis='columns')
mercado = mercado.drop(['empsetorcensitariofaixarendapopulacao'] , axis='columns')
mercado = mercado.drop(['nu_meses_rescencia'] , axis='columns')
mercado = mercado.drop(['vl_potenc_cons_oleo_gas'] , axis='columns')
mercado = mercado.drop(['qt_ex_funcionarios'] , axis='columns')
mercado = mercado.drop(['percent_func_genero_masc'] , axis='columns')
mercado = mercado.drop(['percent_func_genero_fem'] , axis='columns')
mercado = mercado.drop(['fl_spa'] , axis='columns')
mercado = mercado.drop(['dt_situacao'] , axis='columns')

# Não serão considerados os valores relacionados as coligadas
lista_remove = [x for x in mercado.columns if 'colig' in x ]
mercado = mercado.drop(lista_remove , axis='columns')

# Não serão considerados os dados relacionados aos sócios - Apenas a quantidade atual de sócios
lista_remove = [x for x in mercado.columns if 'socio' in x ]
lista_remove.remove('qt_socios')
mercado = mercado.drop(lista_remove , axis='columns')

# Não serão considerados os dados de Ramo, Segmento e Divisão - Consideraremos o dado de setor para agrupamento dos clientes
mercado = mercado.drop(['de_ramo'] , axis='columns')
mercado = mercado.drop(['nm_segmento'] , axis='columns')
mercado = mercado.drop(['nm_divisao'] , axis='columns')

# Não serão consideradas as idades dos funcionários para avaliação do cliente
lista_remove = [x for x in mercado.columns if 'idade_de' in x ]
mercado = mercado.drop(lista_remove, axis='columns')
lista_remove = [x for x in mercado.columns if 'idade_ate' in x ]
mercado = mercado.drop(lista_remove, axis='columns')
lista_remove = [x for x in mercado.columns if 'idade_acima' in x ]
mercado = mercado.drop(lista_remove, axis='columns')

# Não serão consideradas as graus de instrução dos funcionários para avaliação do cliente
lista_remove = [x for x in mercado.columns if 'grau_instrucao_macro' in x ]
mercado = mercado.drop(lista_remove, axis='columns')

# Não serão consideradas as meses de serviço dos funcionários para avaliação do cliente
lista_remove = [x for x in mercado.columns if 'meses_servicos' in x ]
mercado = mercado.drop(lista_remove, axis='columns')

# Não serão considerados os dados de veículos
lista_remove = [x for x in mercado.columns if 'veiculo' in x ]
mercado = mercado.drop(lista_remove, axis='columns')

# Não serão considerados os dados de e-mail e telefone
mercado = mercado.drop(['fl_email'] , axis='columns')
mercado = mercado.drop(['fl_telefone'] , axis='columns')
mercado = mercado.drop(['de_indicador_telefone'] , axis='columns')

# Não serão considerados os dados de impostos
mercado = mercado.drop(['fl_passivel_iss'] , axis='columns')
mercado = mercado.drop(['fl_st_especial'] , axis='columns')

# Não serão considerados os dados de Matriz/Grupo
lista_remove = [x for x in mercado.columns if 'matriz' in x ]
mercado = mercado.drop(lista_remove, axis='columns')

# Não serão considerados os dados de Saude
lista_remove = [x for x in mercado.columns if 'saude' in x ]
mercado = mercado.drop(lista_remove, axis='columns')

# Não serão considerados os dados de Matriz/Grupo
lista_remove = [x for x in mercado.columns if 'grupo' in x ]
mercado = mercado.drop(lista_remove, axis='columns')

In [4]:
# Importando o Dataframe de Portfolio1
portfolio1 = pd.read_csv('estaticos_portfolio1.csv')
portfolio1 = portfolio1.drop(['Unnamed: 0'] , axis='columns')
# Deixando somente o id no portfolio
lista_port1 = list(portfolio1.columns)
lista_port1.remove('id')
portfolio1 = portfolio1.drop(lista_port1, axis='columns')
portfolio1.head()

,id
0,dabe79bec87c88ae04e869bf6bd321ee5e1893cecf6625...
1,32e5f4e10932153a7ba869cb0386e7e02d49d2461046b8...
2,a95d6f30bba445bd3d6b0c5b36f865b38ec01d17336090...
3,6cb309685cea0b6d2988818792ec2e6fcb2bd02e0afa9e...
4,f72aa7fa6787b0a5a1c88885b6120850df8ee0f71adc25...


In [5]:
# Importando o Dataframe de Portfolio2
portfolio2 = pd.read_csv('estaticos_portfolio2.csv')
portfolio2 = portfolio2.drop(['Unnamed: 0'] , axis='columns')
#portfolio2 = pd.merge(portfolio2, mercado, on='id', how='inner')
portfolio2.head()

,id
0,09e95c1a84049001d086470a8f320a19b076f955a89122...
1,dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728...
2,16843c9ffb920170477118933798510d8d5f00f5c56c12...
3,ff045934d90aab184dd15e66daf3b3c97193bda5449298...
4,a0ce528caab2c62497a548d0ebd9e935143ef81ed7dbf0...


In [6]:
# Importando o Dataframe de Portfolio3
portfolio3 = pd.read_csv('estaticos_portfolio3.csv')
portfolio3 = portfolio3.drop(['Unnamed: 0'] , axis='columns')
#portfolio3 = pd.merge(portfolio3, mercado, on='id', how='inner')
portfolio3.head()

,id
0,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...
1,16843c9ffb920170477118933798510d8d5f00f5c56c12...
2,1b8d092daff12bed8b3c2ab93bf5df2921865a68aca185...
3,3aafa3a1f12f483740a52956af100223333d0e3c79bf2c...
4,c60431fbca90067f105570e1e8a684bcd7048205bbf6c4...


## Iniciando as Análises dos Arquivos

Nesse momento avaliaremos os dados do dataframe

In [7]:
# 5 Primeiras Linhas
mercado.head()

,id,de_natureza_juridica,sg_uf,natureza_juridica_macro,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,...,fl_optante_simei,de_nivel_atividade,fl_simples_irregular,nm_meso_regiao,nm_micro_regiao,qt_socios,vl_faturamento_estimado_aux,total,qt_funcionarios,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,False,False,...,False,ALTA,False,LESTE POTIGUAR,NATAL,2.0,3132172.8,26.0,26.0,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVIÇO,1.463014,1 a 5,False,False,False,...,NaN,BAIXA,False,CENTRO NORTE PIAUIENSE,TERESINA,1.0,210000.0,NaN,NaN,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVIÇO,7.093151,5 a 10,False,False,False,...,True,MEDIA,False,CENTRO AMAZONENSE,MANAUS,1.0,50000.0,NaN,NaN,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVIÇO,6.512329,5 a 10,False,False,False,...,False,MEDIA,False,CENTRO AMAZONENSE,MANAUS,1.0,210000.0,NaN,NaN,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,EMPRESARIO INDIVIDUAL,RN,OUTROS,SERVIÇO,3.200000,1 a 5,False,False,False,...,False,ALTA,False,LESTE POTIGUAR,NATAL,1.0,210000.0,NaN,NaN,0


In [8]:
#Verificando Quantidade de Colunas e Observações
mercado.shape

(462298, 25)

In [9]:
#Verificando a quantidade valores nulos por coluna
mercado.isnull().sum()

id                                  0
de_natureza_juridica                0
sg_uf                               0
natureza_juridica_macro             0
setor                            1927
idade_empresa_anos                  0
idade_emp_cat                       0
fl_me                               0
fl_sa                               0
fl_epp                              0
fl_mei                              0
fl_ltda                             0
fl_rm                               0
fl_optante_simples              82713
qt_art                         455708
fl_optante_simei                82713
de_nivel_atividade              11168
fl_simples_irregular             1927
nm_meso_regiao                  58698
nm_micro_regiao                 58698
qt_socios                      115091
vl_faturamento_estimado_aux     27513
total                          386356
qt_funcionarios                358724
qt_filiais                          0
dtype: int64

In [10]:
#Avaliação do mercado que estão sem setor
mercado.groupby('setor').id.count()

setor
AGROPECUARIA          3247
COMERCIO            211224
CONSTRUÇÃO CIVIL     25363
INDUSTRIA            30893
SERVIÇO             189644
Name: id, dtype: int64

In [11]:
mercado.de_natureza_juridica.unique()

array(['SOCIEDADE EMPRESARIA LIMITADA', 'EMPRESARIO INDIVIDUAL',
       'COOPERATIVA', 'ASSOCIACAO PRIVADA', 'ENTIDADE SINDICAL',
       'CONSORCIO DE SOCIEDADES', 'SOCIEDADE DE ECONOMIA MISTA',
       'CONDOMINIO EDILICIO', 'ORGAO PUBLICO DO PODER EXECUTIVO FEDERAL',
       'MUNICIPIO', 'ORGAO DE DIRECAO LOCAL DE PARTIDO POLITICO',
       'SOCIEDADE ANONIMA ABERTA',
       'EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA EMPRESARIA',
       'CANDIDATO A CARGO POLITICO ELETIVO',
       'EMPRESA INDIVIDUAL IMOBILIARIA',
       'ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU DO DISTRITO FEDERAL',
       'SOCIEDADE SIMPLES LIMITADA', 'FUNDACAO PRIVADA',
       'ORGAO PUBLICO DO PODER EXECUTIVO MUNICIPAL',
       'SOCIEDADE ANONIMA FECHADA', 'ORGANIZACAO RELIGIOSA',
       'SOCIEDADE UNIPESSOAL DE ADVOCACIA', 'SOCIEDADE SIMPLES PURA',
       'AUTARQUIA FEDERAL', 'ORGAO PUBLICO DO PODER JUDICIARIO ESTADUAL',
       'EMPRESA PUBLICA', 'SOCIEDADE EMPRESARIA EM NOME COLETIVO',
      

In [12]:
## ONE HOT ENCODE: DE_NIVEL_ATIVIDADE, SETOR_UNIQUE (TROCAR NAN PARA OUTROS), NATUREZA_JURIDICA_MACRO, SG_UF (VERIFICAR NECESSIDADE)
dfDummiesNivelAtividade = pd.get_dummies(mercado['de_nivel_atividade'],prefix='NIVEL_ATIVIDADE_OHE')

In [13]:
## ONE HOT ENCODE: DE_NIVEL_ATIVIDADE, SETOR_UNIQUE (TROCAR NAN PARA OUTROS), NATUREZA_JURIDICA_MACRO, SG_UF (VERIFICAR NECESSIDADE)
dfDummiesSetor = pd.get_dummies(mercado['setor'],prefix='setor_OHE')

In [14]:
## ONE HOT ENCODE: DE_NIVEL_ATIVIDADE, SETOR_UNIQUE (TROCAR NAN PARA OUTROS), NATUREZA_JURIDICA_MACRO, SG_UF (VERIFICAR NECESSIDADE)
dfDummiesNaturezaJuridica = pd.get_dummies(mercado['natureza_juridica_macro'],prefix='NATUREZA_JURIDICA_OHE')

In [15]:
## ONE HOT ENCODE: DE_NIVEL_ATIVIDADE, SETOR_UNIQUE (TROCAR NAN PARA OUTROS), NATUREZA_JURIDICA_MACRO, SG_UF (VERIFICAR NECESSIDADE)
dfDummiesUF = pd.get_dummies(mercado['sg_uf'],prefix='UF_OHE')

In [16]:
#Atualizando DF
mercado = pd.concat([mercado, dfDummiesNivelAtividade], axis=1)
mercado = pd.concat([mercado, dfDummiesSetor], axis=1)
mercado = pd.concat([mercado, dfDummiesNaturezaJuridica], axis=1)
mercado = pd.concat([mercado, dfDummiesUF], axis=1)

In [17]:
mercado.head()

,id,de_natureza_juridica,sg_uf,natureza_juridica_macro,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,...,NATUREZA_JURIDICA_OHE_ENTIDADES SEM FINS LUCRATIVOS,NATUREZA_JURIDICA_OHE_INSTITUICOES EXTRATERRITORIAIS,NATUREZA_JURIDICA_OHE_OUTROS,NATUREZA_JURIDICA_OHE_PESSOAS FISICAS,UF_OHE_AC,UF_OHE_AM,UF_OHE_MA,UF_OHE_PI,UF_OHE_RN,UF_OHE_RO
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,False,False,...,0,0,0,0,0,0,0,0,1,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVIÇO,1.463014,1 a 5,False,False,False,...,0,0,1,0,0,0,0,1,0,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVIÇO,7.093151,5 a 10,False,False,False,...,0,0,1,0,0,1,0,0,0,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVIÇO,6.512329,5 a 10,False,False,False,...,0,0,1,0,0,1,0,0,0,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,EMPRESARIO INDIVIDUAL,RN,OUTROS,SERVIÇO,3.200000,1 a 5,False,False,False,...,0,0,1,0,0,0,0,0,1,0


In [18]:
mercado = mercado.drop(['de_nivel_atividade'] , axis='columns')
mercado = mercado.drop(['setor'] , axis='columns')
mercado = mercado.drop(['natureza_juridica_macro'] , axis='columns')
mercado = mercado.drop(['sg_uf'] , axis='columns')

In [19]:
#Verificando a quantidade valores nulos por coluna
media_qt_art = round(mercado.qt_art.mean(),0)
media_qt_art

6.0

In [20]:
#Atualizando Medina nos valores null da QT_ART
mercado.qt_art.fillna(media_qt_art, inplace=True)

In [21]:
## TROCAR NAN FL_* PARA FALSE
mercado.fl_optante_simei.fillna(False, inplace=True)
mercado.fl_optante_simples.fillna(False, inplace=True)
mercado.fl_simples_irregular.fillna(False, inplace=True)

In [22]:
# ATUALIZANDO QUANTIDADE FUNCIONARIOS E SOCIOS
mercado.qt_funcionarios.fillna(1, inplace=True)
mercado.qt_socios.fillna(1, inplace=True)

In [23]:
mercado.nm_meso_regiao.fillna('OUTRA', inplace=True)
mercado.nm_micro_regiao.fillna('OUTRA', inplace=True)
mercado.isna().sum()

id                                                           0
de_natureza_juridica                                         0
idade_empresa_anos                                           0
idade_emp_cat                                                0
fl_me                                                        0
fl_sa                                                        0
fl_epp                                                       0
fl_mei                                                       0
fl_ltda                                                      0
fl_rm                                                        0
fl_optante_simples                                           0
qt_art                                                       0
fl_optante_simei                                             0
fl_simples_irregular                                         0
nm_meso_regiao                                               0
nm_micro_regiao                                        

In [ ]:
## VERIFICAR IDADE EMPRESA
## VERIFICAR DE_NATUREZA_JURIDICA
## VERIFICAR vl_faturamento_estimado_aux - Muitos valores com NaN
## VERIFICAR total - Muitos valores com NaN